In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import numpy as np
import pandas as pd
import datetime
import math

In [0]:
p = lambda x: f'/content/drive/My Drive/MLfinal/{x}'
train_df = pd.read_csv(p('test.csv'))
weather_df = pd.read_csv(p('weather_test.csv'))
building_metadata_df = pd.read_csv(p('building_metadata.csv'))

In [0]:
def print_density(data, name="Data"):
  print(f"{name} Density:")
  for column in data.columns:
    density = str(100 * (len(data) - np.sum(data[column].isnull())) / len (data))
    print(column + ': ' + density)
  print('\n')

print_density(train_df, "Test Data")
print_density(weather_df, "Weather Data")
print_density(building_metadata_df, "Building Metadata")


Test Data Density:
row_id: 100.0
building_id: 100.0
meter: 100.0
timestamp: 100.0


Weather Data Density:
site_id: 100.0
timestamp: 100.0
air_temperature: 99.9624877814769
cloud_coverage: 49.341191662188045
dew_temperature: 99.88205292829755
precip_depth_1_hr: 65.52194284436396
sea_level_pressure: 92.32983339525254
wind_direction: 95.53821016220428
wind_speed: 99.83408057191706


Building Metadata Density:
site_id: 100.0
building_id: 100.0
primary_use: 100.0
square_feet: 100.0
year_built: 46.58385093167702
floor_count: 24.499654934437544




In [0]:
train_df["timestamp"] = [datetime.datetime.strptime(time_stamp, '%Y-%m-%d %H:%M:%S') for time_stamp in train_df["timestamp"]]

In [0]:
weather_df["timestamp"] = [datetime.datetime.strptime(time_stamp, '%Y-%m-%d %H:%M:%S') for time_stamp in weather_df["timestamp"]]


In [0]:
# convert timestamp column to datetime object
weather_df["year"] = [time_stamp.year for time_stamp in weather_df["timestamp"]]
weather_df["month"] = [time_stamp.month for time_stamp in weather_df["timestamp"]]
weather_df["day"] = [time_stamp.day for time_stamp in weather_df["timestamp"]]
weather_df["hour"] = [time_stamp.hour for time_stamp in weather_df["timestamp"]]

fill in missing weather data

In [0]:
new_weather_df = weather_df.copy()

delete precip_depth_1_hr, cloud_coverage, and 'sea_level_pressure' because missing for entire sites

In [0]:
new_weather_df.drop(columns=['sea_level_pressure', 'cloud_coverage', 'precip_depth_1_hr'], inplace=True)

In [0]:
null_weather_cols = ['air_temperature', 'dew_temperature', 'wind_direction', 'wind_speed']

In [0]:
count = 0
for site in new_weather_df.site_id.unique():
  site_data = new_weather_df.loc[weather_df.site_id == site, ]
  site_null_rows = site_data[site_data.isnull().any(axis=1)]
  for column in null_weather_cols:
    null_indices = site_null_rows[site_null_rows[column].isnull()].index
    for index, row in site_null_rows.iterrows():
      if index in null_indices:
        max_index = site_data.index.max()
        min_index = site_data.index.min()
        prev_day = new_weather_df.loc[index-1, column] if index !=min_index else float('NaN')
        next_day = new_weather_df.loc[index+1, column] if index != max_index else float('NaN')
      # new_value = prev_day if if math.isnan(next_day) else np.mean([prev_day, next_day])
        # weather_df.loc[index, column] = prev_day if math.isnan(next_day) else np.mean([prev_day, next_day])
        if math.isnan(prev_day):
          if math.isnan(next_day): # take avg of that month, and hour, and site
            count+= 1
            new_val = np.mean(site_data.loc[(site_data.month == row.month) & (site_data.hour == row.hour), column])
          # print(row)
          else: # future not prev
            new_val = next_day
        elif math.isnan(next_day): # no future
          new_val = prev_day
        else: # both
          new_val = np.mean([prev_day, next_day])
        new_weather_df.loc[index, column] = new_val
    print('done ', site, column)

done  0 air_temperature
done  0 dew_temperature
done  0 wind_direction
done  0 wind_speed
done  1 air_temperature
done  1 dew_temperature
done  1 wind_direction
done  1 wind_speed
done  2 air_temperature
done  2 dew_temperature
done  2 wind_direction
done  2 wind_speed
done  3 air_temperature
done  3 dew_temperature
done  3 wind_direction
done  3 wind_speed
done  4 air_temperature
done  4 dew_temperature
done  4 wind_direction
done  4 wind_speed
done  5 air_temperature
done  5 dew_temperature
done  5 wind_direction
done  5 wind_speed
done  6 air_temperature
done  6 dew_temperature
done  6 wind_direction
done  6 wind_speed
done  7 air_temperature
done  7 dew_temperature
done  7 wind_direction
done  7 wind_speed
done  8 air_temperature
done  8 dew_temperature
done  8 wind_direction
done  8 wind_speed
done  9 air_temperature
done  9 dew_temperature
done  9 wind_direction
done  9 wind_speed
done  10 air_temperature
done  10 dew_temperature
done  10 wind_direction
done  10 wind_speed
done  

Merge the dataset

In [0]:
building_meter_data = train_df.merge(building_metadata_df, how = 'left', on='building_id')

In [0]:
merged_data = building_meter_data.merge(new_weather_df, how = 'left', on=['timestamp', 'site_id'])

In [0]:
null_weather_times = merged_data.loc[merged_data.air_temperature.isnull(), ['timestamp', 'site_id']]
null_weather_times.drop_duplicates(subset=None, keep='first', inplace=True)

In [0]:
null_weather_times

,timestamp,site_id
41498571,2017-01-18 09:00:00,1
41498634,2017-01-18 19:00:00,1
41498697,2017-08-08 03:00:00,1
41498760,2017-08-15 18:00:00,1
41498823,2017-08-15 19:00:00,1
...,...,...
41696822,2017-12-25 04:00:00,11
41696836,2017-12-25 07:00:00,11
41696850,2018-05-09 05:00:00,15
41697100,2018-05-09 06:00:00,15


In [0]:
final_weather = new_weather_df.copy()

In [0]:
new_null_weather_times = []
count = 0
for index, timeset in null_weather_times.iterrows():
  time_val = timeset.timestamp
  site = timeset.site_id
  prev_hour = new_weather_df.loc[(new_weather_df.timestamp == (null_weather_times.loc[index, 'timestamp'] - np.timedelta64(1, 'h'))) & (new_weather_df.site_id ==site) ,] if time_val.hour != 0 else float('NaN')
  next_hour = new_weather_df.loc[(new_weather_df.timestamp == (null_weather_times.loc[index, 'timestamp'] + np.timedelta64(1, 'h'))) & (new_weather_df.site_id ==site) ,] if time_val.hour != 11 else float('NaN')
  if not isinstance(prev_hour, float):  
    prev_hour = float('NaN') if prev_hour.empty else prev_hour
  if not isinstance(next_hour, float):
    next_hour = float('NaN') if next_hour.empty else next_hour
  if isinstance(prev_hour, float): # no prev row
    if isinstance(next_hour, float):
     # print(index, "NO SURROUNDING TIME STAMP")
     # new_count+= 1
      new_null_weather_times.append([time_val, site])
    else:
      new_row = next_hour.copy()
     # print(index, "prev_only")
      new_row.timestamp.iloc[0] = time_val
      new_row.month.iloc[0] = time_val.month
      new_row.day.iloc[0] = time_val.day
      new_row.hour.iloc[0] = time_val.hour
      count+= 1
     # print(new_row, " prev_only")
  elif isinstance(next_hour, float): # prev row, no future row
    new_row = prev_hour.copy()
   # print(index, "future only")
    new_row.timestamp.iloc[0] = time_val
    new_row.month.iloc[0] = time_val.month
    new_row.day.iloc[0] = time_val.day
    new_row.hour.iloc[0] = time_val.hour
    count+= 1
  #  print(new_row, " future_only")
  else: # average both rows
    # join two rows
  #  print(index, "both")
    both_rows = prev_hour.append(next_hour)
    test_means = np.mean(both_rows.loc[:,['air_temperature', 'dew_temperature', 'wind_direction', 'wind_speed']]).values
    new_row = [site, time_val]
    for value in test_means:
      new_row.append(value)
    new_row.append(time_val.year)
    new_row.append(time_val.month)
    new_row.append(time_val.day)
    new_row.append(time_val.hour)
    new_row = pd.DataFrame([new_row], columns=new_weather_df.columns)
   # print(new_row, " both") 
    count+= 1
  final_weather = final_weather.append(new_row)
  


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


Sort Data Frame then try again

In [0]:
final_weather = final_weather.sort_values(by=['site_id', 'timestamp'])

In [0]:
final_weather.drop_duplicates(subset=None, keep='first', inplace=True)

In [0]:
final_weather.shape[0]-new_weather_df.shape[0]

833

In [0]:
count

833

In [0]:
new_null_weather_times = pd.DataFrame(new_null_weather_times, columns=null_weather_times.columns)

In [0]:
new_null_weather_times

,timestamp,site_id
0,2017-08-15 19:00:00,1
1,2018-07-08 16:00:00,1
2,2018-07-08 17:00:00,1
3,2018-08-08 20:00:00,1
4,2018-09-15 18:00:00,1
...,...,...
2239,2017-12-18 18:00:00,11
2240,2017-12-18 19:00:00,11
2241,2017-12-18 20:00:00,11
2242,2017-12-18 21:00:00,11


In [0]:
appended_weather = final_weather.copy()

In [0]:
third_null_weather_times = []
new_count = 0
count = 0
for index, timeset in new_null_weather_times.iterrows():
  time_val = timeset.timestamp
  site = timeset.site_id
  prev_hour = final_weather.loc[(final_weather.timestamp == (new_null_weather_times.loc[index, 'timestamp'] - np.timedelta64(1, 'h'))) & (final_weather.site_id ==site) ,] if time_val.hour != 0 else float('NaN')
  next_hour = final_weather.loc[(final_weather.timestamp == (new_null_weather_times.loc[index, 'timestamp'] + np.timedelta64(1, 'h'))) & (final_weather.site_id ==site) ,] if time_val.hour != 11 else float('NaN')
  if not isinstance(prev_hour, float):  
    prev_hour = float('NaN') if prev_hour.empty else prev_hour
  if not isinstance(next_hour, float):
    next_hour = float('NaN') if next_hour.empty else next_hour
  if isinstance(prev_hour, float): # no prev row
    if isinstance(next_hour, float):
      # print(index, "NO SURROUNDING TIME STAMP")
      # take the average of mnth and day for that site
      site_data = final_weather.loc[(final_weather.site_id == site) & (final_weather.month == time_val.month) & (final_weather.day == time_val.day),]
      new_count+= 1
      count+= 1
      third_null_weather_times.append([time_val, site])
      test_means = np.mean(site_data.loc[:,['air_temperature', 'dew_temperature', 'wind_direction', 'wind_speed']]).values
      new_row_array = [site, time_val]
      for value in test_means:
        new_row_array.append(value)
      new_row_array.append(time_val.year)
      new_row_array.append(time_val.month)
      new_row_array.append(time_val.day)
      new_row_array.append(time_val.hour)
      new_row = pd.DataFrame([new_row_array], columns=appended_weather.columns)
      print(new_row)
    else:
      new_row = next_hour
     # print(index, "prev_only")
      new_row.iloc[0].timestamp = time_val
      new_row.iloc[0].month = time_val.month
      new_row.iloc[0].day = time_val.day
      new_row.iloc[0].hour = time_val.hour
      count+= 1
     # print(new_row, " prev_only")
  elif isinstance(next_hour, float): # prev row, no future row
    new_row = prev_hour
   # print(index, "future only")
    new_row.iloc[0].timestamp = time_val
    new_row.iloc[0].month = time_val.month
    new_row.iloc[0].day = time_val.day
    new_row.iloc[0].hour = time_val.hour
    count+= 1
  #  print(new_row, " future_only")
  else: # average both rows
    # join two rows
  #  print(index, "both")
    both_rows = prev_hour.append(next_hour)
    test_means = np.mean(both_rows.loc[:,['air_temperature', 'dew_temperature', 'wind_direction', 'wind_speed']]).values
    new_row_array = [site, time_val]
    for value in test_means:
      new_row_array.append(value)
    new_row_array.append(time_val.year)
    new_row_array.append(time_val.month)
    new_row_array.append(time_val.day)
    new_row_array.append(time_val.hour)
    new_row = pd.DataFrame([new_row_array], columns=appended_weather.columns)
   # print(new_row, " both") 
    count+= 1
  appended_weather = appended_weather.append(new_row)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


   site_id           timestamp  air_temperature  ...  month  day  hour
0        1 2018-09-15 19:00:00        13.351111  ...      9   15    19

[1 rows x 10 columns]
   site_id           timestamp  air_temperature  ...  month  day  hour
0        1 2018-09-17 05:00:00        14.251613  ...      9   17     5

[1 rows x 10 columns]
   site_id           timestamp  air_temperature  ...  month  day  hour
0        1 2018-09-17 06:00:00        14.251613  ...      9   17     6

[1 rows x 10 columns]
   site_id           timestamp  air_temperature  ...  month  day  hour
0        1 2018-09-17 07:00:00        14.251613  ...      9   17     7

[1 rows x 10 columns]
   site_id           timestamp  air_temperature  ...  month  day  hour
0        1 2018-09-17 08:00:00        14.251613  ...      9   17     8

[1 rows x 10 columns]
   site_id           timestamp  air_temperature  ...  month  day  hour
0        1 2018-09-17 09:00:00        14.251613  ...      9   17     9

[1 rows x 10 columns]
   site_id

In [0]:
appended_weather.drop_duplicates(subset=None, keep='first', inplace=True)

In [0]:
appended_weather = appended_weather.sort_values(by=['site_id', 'timestamp'])

In [0]:
print(appended_weather.shape[0]-final_weather.shape[0])
print(count)
print(new_count)
print(count+new_count)

2041
2244
2017
4261


In [0]:
len(third_null_weather_times)

2017

Merge again with filled in weather timestamps

In [0]:
new_merged_data = building_meter_data.merge(appended_weather, how = 'left', on=['timestamp', 'site_id'])
new_merged_data.loc[new_merged_data.air_temperature.isnull(), ].shape

(19197, 17)

In [0]:
final_null_weather_times = new_merged_data.loc[new_merged_data.air_temperature.isnull(), ['timestamp', 'site_id']]
final_null_weather_times.drop_duplicates(subset=None, keep='first', inplace=True)

In [0]:
final_null_weather_times

,timestamp,site_id
41499327,2018-07-08 16:00:00,1
41499390,2018-07-08 17:00:00,1
41499957,2018-09-15 18:00:00,1
41500083,2018-09-15 20:00:00,1
41500272,2018-09-16 00:00:00,1
...,...,...
41693434,2017-12-07 20:00:00,11
41693840,2017-12-09 01:00:00,11
41693938,2017-12-09 11:00:00,11
41694008,2017-12-10 14:00:00,11


In [0]:
missing_months = [(time.site_id, time.timestamp.month, time.timestamp.hour)for index, time in final_null_weather_times.iterrows()]

In [0]:
set(missing_months)

{(1, 7, 16),
 (1, 7, 17),
 (1, 9, 0),
 (1, 9, 4),
 (1, 9, 8),
 (1, 9, 15),
 (1, 9, 16),
 (1, 9, 18),
 (1, 9, 20),
 (1, 9, 23),
 (1, 11, 18),
 (1, 11, 20),
 (5, 4, 12),
 (5, 4, 18),
 (5, 7, 16),
 (5, 7, 17),
 (5, 8, 2),
 (5, 8, 3),
 (5, 9, 1),
 (5, 9, 4),
 (5, 9, 6),
 (5, 9, 8),
 (5, 9, 9),
 (5, 9, 18),
 (5, 9, 20),
 (5, 9, 23),
 (5, 11, 18),
 (5, 11, 20),
 (6, 11, 21),
 (6, 11, 22),
 (7, 1, 3),
 (7, 1, 5),
 (7, 1, 7),
 (7, 1, 11),
 (7, 1, 13),
 (7, 2, 11),
 (7, 2, 16),
 (7, 2, 17),
 (7, 3, 13),
 (7, 3, 23),
 (7, 4, 11),
 (7, 4, 12),
 (7, 4, 14),
 (7, 4, 16),
 (7, 4, 17),
 (7, 4, 19),
 (7, 6, 0),
 (7, 6, 4),
 (7, 6, 6),
 (7, 6, 11),
 (7, 7, 9),
 (7, 7, 23),
 (7, 8, 3),
 (7, 8, 9),
 (7, 8, 11),
 (7, 8, 14),
 (7, 8, 23),
 (7, 9, 4),
 (7, 9, 8),
 (7, 9, 9),
 (7, 9, 11),
 (7, 9, 16),
 (7, 9, 17),
 (7, 9, 22),
 (7, 9, 23),
 (7, 10, 2),
 (7, 10, 4),
 (7, 10, 16),
 (7, 10, 18),
 (7, 11, 10),
 (7, 11, 13),
 (7, 11, 18),
 (7, 11, 20),
 (7, 12, 1),
 (7, 12, 11),
 (7, 12, 14),
 (7, 12, 20),
 (7, 1

In [0]:
last_shot = appended_weather.copy()

In [0]:
third_null_weather_times = []
new_count = 0
count = 0
for index, timeset in final_null_weather_times.iterrows():
  time_val = timeset.timestamp
  site = timeset.site_id
  prev_hour = appended_weather.loc[(appended_weather.timestamp == (final_null_weather_times.loc[index, 'timestamp'] - np.timedelta64(1, 'h'))) & (appended_weather.site_id ==site) ,] if time_val.hour != 0 else float('NaN')
  next_hour = appended_weather.loc[(appended_weather.timestamp == (final_null_weather_times.loc[index, 'timestamp'] + np.timedelta64(1, 'h'))) & (appended_weather.site_id ==site) ,] if time_val.hour != 11 else float('NaN')
  if not isinstance(prev_hour, float):  
    prev_hour = float('NaN') if prev_hour.empty else prev_hour
  if not isinstance(next_hour, float):
    next_hour = float('NaN') if next_hour.empty else next_hour
  if isinstance(prev_hour, float): # no prev row
    if isinstance(next_hour, float):
      # print(index, "NO SURROUNDING TIME STAMP")
      # take the average of mnth and day for that site
      site_data = appended_weather.loc[(appended_weather.site_id == site) & (appended_weather.month == time_val.month) & (appended_weather.day == time_val.day),]
      new_count+= 1
      count+= 1
      final_null_weather_times.append([time_val, site])
      test_means = np.mean(site_data.loc[:,['air_temperature', 'dew_temperature', 'wind_direction', 'wind_speed']]).values
      new_row_array = [site, time_val]
      for value in test_means:
        new_row_array.append(value)
      new_row_array.append(time_val.year)
      new_row_array.append(time_val.month)
      new_row_array.append(time_val.day)
      new_row_array.append(time_val.hour)
      new_row = pd.DataFrame([new_row_array], columns=last_shot.columns)
      print(new_row)
    else:
      new_row = next_hour
     # print(index, "prev_only")
      new_row.iloc[0].timestamp = time_val
      new_row.iloc[0].month = time_val.month
      new_row.iloc[0].day = time_val.day
      new_row.iloc[0].hour = time_val.hour
      count+= 1
     # print(new_row, " prev_only")
  elif isinstance(next_hour, float): # prev row, no future row
    new_row = prev_hour
   # print(index, "future only")
    new_row.iloc[0].timestamp = time_val
    new_row.iloc[0].month = time_val.month
    new_row.iloc[0].day = time_val.day
    new_row.iloc[0].hour = time_val.hour
    count+= 1
  #  print(new_row, " future_only")
  else: # average both rows
    # join two rows
  #  print(index, "both")
    both_rows = prev_hour.append(next_hour)
    test_means = np.mean(both_rows.loc[:,['air_temperature', 'dew_temperature', 'wind_direction', 'wind_speed']]).values
    new_row_array = [site, time_val]
    for value in test_means:
      new_row_array.append(value)
    new_row_array.append(time_val.year)
    new_row_array.append(time_val.month)
    new_row_array.append(time_val.day)
    new_row_array.append(time_val.hour)
    new_row = pd.DataFrame([new_row_array], columns=last_shot.columns)
   # print(new_row, " both") 
    count+= 1
  last_shot = last_shot.append(new_row)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [0]:
last_shot.drop_duplicates(subset=None, keep='first', inplace=True)

In [0]:
last_shot = last_shot.sort_values(by=['site_id', 'timestamp'])

In [0]:
print(last_shot.shape[0]-appended_weather.shape[0])
print(count)
print(new_count)
print(count+new_count)

164
203
0
203


In [0]:
len(third_null_weather_times)

0

FINAL MERGE ATTEMPT

In [0]:
final_merge = building_meter_data.merge(manually_adjusted, how = 'left', on=['timestamp', 'site_id'])

In [0]:
final_merge.head()

Manually filled in remaining missing timestamps (holidays like Thanksgiving and MLK day where several sites were missing the same timestamp across the board, but these values were lost and did not make a huge impact as var as the density levels of the final merged dataset: .95 --> .99)

In [0]:
for site in manually_adjusted.site_id.unique():
  site_data = manually_adjusted.loc[manually_adjusted.site_id == site]
  print(site_data.shape)

(8784, 10)
(8784, 10)
(8784, 10)
(8784, 10)
(8784, 10)
(8784, 10)
(8784, 10)
(8784, 10)
(8784, 10)
(8784, 10)
(8784, 10)
(8784, 10)
(8784, 10)
(8784, 10)
(8784, 10)
(8780, 10)


In [0]:
site_0 = last_shot.loc[last_shot.site_id == 15,]

In [0]:
for month in site_0.month.unique():
  print(site_0.loc[site_0.month == month,].shape[0])


744
696
740
720
744
718
744
742
720
744
720
744


In [0]:
count = 1
for day in site_0.loc[site_0.month == 2,].day.unique():
  day_data = site_0.loc[(site_0.month == 2) & (site_0.day == day),]
  print(count, day_data.shape)
  count+=1

1 (24, 10)
2 (24, 10)
3 (24, 10)
4 (24, 10)
5 (24, 10)
6 (24, 10)
7 (24, 10)
8 (24, 10)
9 (24, 10)
10 (24, 10)
11 (24, 10)
12 (24, 10)
13 (24, 10)
14 (24, 10)
15 (22, 10)
16 (24, 10)
17 (24, 10)
18 (24, 10)
19 (24, 10)
20 (24, 10)
21 (24, 10)
22 (24, 10)
23 (24, 10)
24 (24, 10)
25 (24, 10)
26 (24, 10)
27 (24, 10)
28 (24, 10)
29 (24, 10)


In [0]:
site_0.loc[(site_0.month == 2) & (site_0.day == 15),]

,site_id,timestamp,air_temperature,dew_temperature,wind_direction,wind_speed,year,month,day,hour
9864,1,2016-02-15 00:00:00,1.70,-1.3,350.0,3.6,2016,2,15,0
9865,1,2016-02-15 01:00:00,1.10,-1.2,340.0,3.6,2016,2,15,1
9866,1,2016-02-15 02:00:00,0.80,-1.4,340.0,3.1,2016,2,15,2
9867,1,2016-02-15 03:00:00,0.50,-1.5,340.0,2.6,2016,2,15,3
9868,1,2016-02-15 04:00:00,0.60,-1.1,360.0,3.6,2016,2,15,4
9869,1,2016-02-15 05:00:00,0.20,-1.3,330.0,2.6,2016,2,15,5
9870,1,2016-02-15 06:00:00,0.50,-0.9,340.0,3.1,2016,2,15,6
9871,1,2016-02-15 07:00:00,0.90,-0.4,330.0,3.6,2016,2,15,7
9872,1,2016-02-15 08:00:00,2.00,-0.2,350.0,4.1,2016,2,15,8
9873,1,2016-02-15 09:00:00,2.60,-0.2,350.0,4.1,2016,2,15,9


In [0]:
manually_adjusted = last_shot.copy()

In [0]:
seven_done = manually_adjusted.copy()

In [0]:
manually_add = last_shot.loc[(last_shot.site_id == 1) & (last_shot.month == 2) & ((last_shot.day == 15))& ((last_shot.hour == 14) | (last_shot.hour == 17)),]

In [0]:
manually_add

,site_id,timestamp,air_temperature,dew_temperature,wind_direction,wind_speed,year,month,day,hour
9877,1,2016-02-15 14:00:00,6.0,-1.0,360.0,5.1,2016,2,15,14
9878,1,2016-02-15 17:00:00,4.8,-1.1,330.0,3.1,2016,2,15,17


In [0]:
time_val = pd.Timestamp(2016, 2, 15, 16)

In [0]:
test_means = np.mean(manually_add.loc[:,['air_temperature', 'dew_temperature', 'wind_direction', 'wind_speed']]).values
new_row_array = [1, time_val]
for value in test_means:
  new_row_array.append(value)
new_row_array.append(time_val.year)
new_row_array.append(time_val.month)
new_row_array.append(time_val.day)
new_row_array.append(time_val.hour)
new_row = pd.DataFrame([new_row_array], columns=last_shot.columns)

In [0]:
new_row

,site_id,timestamp,air_temperature,dew_temperature,wind_direction,wind_speed,year,month,day,hour
0,1,2016-02-15 16:00:00,5.4,-1.05,345.0,4.1,2016,2,15,16


In [0]:
manually_adjusted = manually_adjusted.append(new_row)

In [0]:
manually_adjusted.tail(25)

,site_id,timestamp,air_temperature,dew_temperature,wind_direction,wind_speed,year,month,day,hour
139771,15,2016-12-31 22:00:00,2.20,-6.70,170.0,4.60,2016,12,31,22
139772,15,2016-12-31 23:00:00,1.70,-5.60,180.0,8.80,2016,12,31,23
0,15,2016-06-28 15:00:00,24.20,18.05,240.0,4.10,2016,6,28,15
0,15,2016-06-28 16:00:00,24.20,18.05,240.0,4.10,2016,6,28,16
0,15,2016-08-18 22:00:00,24.10,19.00,330.0,3.35,2016,8,18,22
0,15,2016-08-18 23:00:00,24.10,19.00,330.0,3.35,2016,8,18,23
0,12,2016-02-18 17:00:00,4.60,1.40,240.0,5.80,2016,2,18,17
0,12,2016-02-18 18:00:00,4.60,1.40,240.0,5.80,2016,2,18,18
0,12,2016-11-23 00:00:00,0.95,-0.30,310.0,4.50,2016,11,23,0
0,11,2016-01-06 11:00:00,-11.00,-12.15,204.0,3.40,2016,1,6,11


In [0]:
manually_adjusted.iloc[-6]

site_id                             12
timestamp          2016-11-23 00:00:00
air_temperature                   0.95
dew_temperature                   -0.3
wind_direction                     310
wind_speed                         4.5
year                              2016
month                               11
day                                 23
hour                                 0
Name: 0, dtype: object

In [0]:
last_shot.to_csv(p('updated_weather_test.csv'))